In [20]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os


import warnings
warnings.filterwarnings("ignore")

In [8]:
# Define params and helpers
xlsx_path = './ODIR_dataset/ODIR-5K_Training_Annotations(Updated)_V2.xlsx'
data_dir = './ODIR_dataset/ODIR-5K_Training_Dataset/'

classes = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
img_size = (160, 160)

In [9]:
df = pd.read_excel(xlsx_path)
df[:5]

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1


In [10]:
def get_input(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return(img)

In [11]:
def get_output(df):
    labels = df[classes].values.tolist()
    return(labels)

In [12]:
def preprocess_input(image):
    img_resized = cv2.resize(image, img_size)
    return(img_resized)

In [13]:
def image_generator(df, batch_size=64):
    df = df.sample(frac=1).reset_index(drop=True)
    img_path_left_list = df['Left-Fundus'].values.tolist()
    img_path_right_list = df['Right-Fundus'].values.tolist()
    labels = get_output(df[classes])
    n_data = len(df)
    count = 0
    
    while True:
        # Select files (paths/indices) for the batch
        
        batch_input_left = []
        batch_input_right = []
        batch_output = [] 
          
        # Read in each input, perform preprocessing and get labels
        for i in range(batch_size):
            idx = count % n_data
            img_path_left = os.path.join(data_dir, img_path_left_list[idx])
            img_path_right = os.path.join(data_dir, img_path_right_list[idx])
            input_left = get_input(img_path_left)
            input_right = get_input(img_path_right)
            output = labels[idx]
            
            input_left = preprocess_input(image=input_left)
            input_right = preprocess_input(image=input_right)
            batch_input_left += [input_left]
            batch_input_right += [input_right]
            batch_output += [output]
        # Return a tuple of (input,output) to feed the network
        batch_x_left = np.array(batch_input_left)
        batch_x_right = np.array(batch_input_right)
        batch_y = np.array(batch_output)
        
        yield([batch_x_left, batch_x_right], batch_y )

In [14]:
def create_model():
    # left head
    base_model_left = ResNet50(include_top=False, pooling='avg', weights='imagenet')
    for layer in base_model_left.layers:
        layer._name = layer.name + '_l'
        layer.trainable = True
    x_l = base_model_left.output
    
    # right head
    base_model_right = ResNet50(include_top=False, pooling='avg', weights='imagenet')
    
    for layer in base_model_right.layers:
        layer._name = layer.name + '_r'
        layer.trainable = True
    x_r = base_model_right.output
    
    x = keras.layers.concatenate([x_l, x_r])
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(512)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.Dense(512)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    out = keras.layers.Dense(len(classes), activation='sigmoid')(x)
    model = keras.models.Model(inputs=[base_model_left.input, base_model_right.input], outputs=out)
    return model

In [15]:
model = create_model()

W1104 18:19:08.381942 20084 deprecation.py:506] From C:\Users\Natthawat\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_l (InputLayer)          [(None, None, None,  0                                            
__________________________________________________________________________________________________
input_2_r (InputLayer)          [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad_l (ZeroPadding2D)     (None, None, None, 3 0           input_1_l[0][0]                  
__________________________________________________________________________________________________
conv1_pad_r (ZeroPadding2D)     (None, None, None, 3 0           input_2_r[0][0]                  
______________________________________________________________________________________________

In [17]:
def binary_focal_loss(gamma=2., alpha=.25):

    def binary_focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [18]:
loss = 'binary_crossentropy'

model.compile(loss=binary_focal_loss(), optimizer=keras.optimizers.Adam(1e-5), metrics=['accuracy'])

W1104 18:19:08.971496 20084 deprecation.py:323] From <ipython-input-17-26f28ccc265d>:4: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
er = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint('./snapshots/baseline/model.h5', save_best_only=True)
reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7)

callbacks = [er, checkpoint, reduce]

In [29]:
batch_size = 8
epoch = 200

# Train test split
train, test = train_test_split(df, test_size=0.2, random_state=85)

train_img_gen = image_generator(train, batch_size)
test_img_gen = image_generator(test, batch_size)

In [32]:
STEP_PER_EPOCH = len(train) // batch_size
STEP_PER_EPOCH_VAL = len(test) // batch_size
model.fit_generator(train_img_gen,
                    steps_per_epoch=STEP_PER_EPOCH,
                    validation_data=test_img_gen,
                    validation_steps=STEP_PER_EPOCH_VAL,
                    callbacks=callbacks,
                    epochs=epoch)

Epoch 1/200
350/350 [==============================] - 522s 1s/step - loss: 10.9779 - acc: 0.2407 - val_loss: 0.3401 - val_acc: 1.0000
Epoch 2/200
350/350 [==============================] - 410s 1s/step - loss: 5.7195 - acc: 0.5696 - val_loss: 4.4375 - val_acc: 0.0000e+00
Epoch 3/200
350/350 [==============================] - 405s 1s/step - loss: 3.1245 - acc: 0.8221 - val_loss: 13.7005 - val_acc: 0.0000e+00
Epoch 4/200
350/350 [==============================] - 399s 1s/step - loss: 1.8878 - acc: 0.9293 - val_loss: 10.0246 - val_acc: 0.0000e+00
Epoch 5/200
350/350 [==============================] - 397s 1s/step - loss: 1.2266 - acc: 0.9736 - val_loss: 14.4571 - val_acc: 0.0000e+00
Epoch 6/200
350/350 [==============================] - 403s 1s/step - loss: 0.8521 - acc: 0.9889 - val_loss: 9.7240 - val_acc: 0.0000e+00
Epoch 7/200
350/350 [==============================] - 405s 1s/step - loss: 0.5842 - acc: 0.9936 - val_loss: 12.4067 - val_acc: 0.0000e+00
Epoch 8/200
350/350 [============

In [10]:
model = load_model = './snapshots/baseline/model.h5'